In [44]:
!pip install dash==2.13.0
!pip install dash_core_components==2.0.0
!pip install dash_html_components==2.0.0
!pip install plotly==5.15.0

In [45]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

In [46]:
import pandas as pd
import os

# Define file path and URL
file_path = 'spacex_launch_data.csv'  # Store in the current directory
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'

# Download the file if it doesn't exist
if not os.path.exists(file_path):
    import urllib.request
    print(f"Downloading file from {url} to {file_path}...")
    urllib.request.urlretrieve(url, file_path)
    print("Download complete.")

# Load data into a pandas DataFrame
spacex_df = pd.read_csv(file_path)

# Proceed with the filtering operation
filtered_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()

In [47]:
filtered_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()

In [48]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import os
import urllib.request

# Download the data file if it doesn't exist
data_file = 'spacex_launch_data.csv'
data_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'

if not os.path.exists(data_file):
    print(f"Downloading data file from {data_url}...")
    urllib.request.urlretrieve(data_url, data_file)
    print("Download complete.")

# Load the data into a pandas DataFrame
spacex_df = pd.read_csv(data_file)

# Define the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Add a Launch Site Drop-down Input Component
    dcc.Dropdown(id='site-dropdown',
                options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                ],
                value='ALL',
                placeholder="Select a Launch Site here",
                searchable=True),
    html.Br(),

    # TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # TASK 3: Add a Range Slider to Select Payload
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[0, 10000]),
    html.Br(),

    # TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Define your callback functions here (update_pie_chart, update_scatter_chart)
# ... (Your existing callback functions) ...

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [50]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px

# ... rest of your code

# Create the range slider component
payload_slider = dcc.RangeSlider(
    id='payload-slider',
    min=0,
    max=10000,
    step=1000,
    value=[min(spacex_df['Payload Mass (kg)']), max(spacex_df['Payload Mass (kg)'])],  # Set initial range to min and max payload
    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}  # Add marks for better readability
)

# Add the range slider to the app layout
app.layout = html.Div([
    # ... your dropdown component
    payload_slider,
    # ... your pie chart component
    # ... other components of your app layout
])

In [52]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px

# ... rest of your code

# Callback function to generate scatter plot based on selected site and payload range
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_plot(entered_site, payload_range):
    """
    Generates a scatter plot to visualize launch success vs. payload mass with booster version coloring.

    Args:
        entered_site (str): The selected launch site from the dropdown menu.
        payload_range (list): A list containing the minimum and maximum payload values.

    Returns:
        px.Figure: A plotly scatter plot figure.
    """

    # Filter data based on selected site and payload range
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                              (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    else:
        filtered_df = spacex_df[(spacex_df['launch_site'] == entered_site) &
                              (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                              (spacex_df['Payload Mass (kg)'] <= payload_range[1])]

    # Create scatter plot with color-coded booster versions
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=f"Launch Outcome vs. Payload Mass for {entered_site if entered_site != 'ALL' else 'All Sites'}",
    )

    return fig

# Add the scatter plot to the app layout
app.layout = html.Div([
    # ... your dropdown component
    # ... your range slider component
    dcc.Graph(id='success-payload-scatter-chart'),
    # ... other components of your app layout
])

In [53]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import os
import urllib.request

# Download the data file if it doesn't exist
data_file = 'spacex_launch_data.csv'
data_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'

if not os.path.exists(data_file):
    print(f"Downloading data file from {data_url}...")
    urllib.request.urlretrieve(data_url, data_file)
    print("Download complete.")

# Load the data into a pandas DataFrame
spacex_df = pd.read_csv(data_file)

# Create the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),

    # TASK 1: Add a Launch Site Drop-down Input Component
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                 ],
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True),
    html.Br(),

    # TASK 2: Add a pie chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    # TASK 3: Add a Range Slider to Select Payload
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000',
                           7500: '7500', 10000: '10000'},
                    value=[0, 10000]),
    html.Br(),

    # TASK 4: Add a scatter chart
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


# Callback for pie chart
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class',
                   names='Launch Site',
                   title='Total Success Launches By Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class',
                   title='Total Success Launches for site ' + entered_site)
    return fig

# Callback for scatter chart
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_plot(entered_site, payload_range):

    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                              (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == entered_site) &
                              (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                              (spacex_df['Payload Mass (kg)'] <= payload_range[1])]

    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=f"Launch Outcome vs. Payload Mass for {entered_site if entered_site != 'ALL' else 'All Sites'}",
    )

    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>